## Predict Product Prices

### Week 7 Day 1

An introduction to LoRA and QLoRA



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## A reminder of 2 important pro-tips for using Colab:

**Pro-tip 1:**

The top of every colab has some pip installs. You may receive errors from pip when you run this, such as:

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

These pip compatibility errors can be safely ignored; and while it's tempting to try to fix them by changing version numbers, that will actually introduce real problems!

**Pro-tip 2:**

In the middle of running a Colab, you might get an error like this:

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

This is a super-misleading error message! Please don't try changing versions of packages...

This actually happens because Google has switched out your Colab runtime, perhaps because Google Colab was too busy. The solution is:

1. Runtime menu >> Disconnect and delete runtime
2. Reload the colab from fresh and Edit menu >> Clear All Outputs
3. Connect to a new T4 using the button at the top right
4. Select "View resources" from the menu on the top right to confirm you have a GPU
5. Rerun the cells in the colab, from the top down, starting with the pip installs

And all should work great - otherwise, ask me!

In [ ]:
# pip installs
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0
!pip install -q datasets requests peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 708.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 112.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 619.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
FINETUNED_MODEL = f"ed-donner/pricer-2024-09-13_13.04.39"

# Hyperparameters for QLoRA Fine-Tuning

LORA_R = 32
LORA_ALPHA = 64
# Setting up the target modules for LoRA adaptation.
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

### Log in to HuggingFace

If you don't already have a HuggingFace account, visit https://huggingface.co to sign up and create a token.

Then select the Secrets for this Notebook by clicking on the key icon in the left, and add a new secret called `HF_TOKEN` with the value as your token.

In [ ]:
# Log in to HuggingFace
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

## Trying out different Quantization



In [ ]:
# Load the Base Model without quantization
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")

In [ ]:
# This helps in understanding on how much memory the base mode is using us.
# This will be around - 32GB (32 bit floats for each 8B parameters)
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

NameError: name 'base_model' is not defined

In [2]:
base_model

NameError: name 'base_model' is not defined

## Restart your session!

In order to load the next model and clear out the cache of the last model, you'll now need to go to Runtime >> Restart session and run the initial cells (installs and imports and HuggingFace login) again.

This is to clean out the GPU.

In [ ]:
# Load the Base Model using 8 bit

# Bites and Bytes configuration helps in specfiying what kind of quantization config we want + defining the reduction in precision of the model.
quant_config = BitsAndBytesConfig(load_in_8bit=True)

# We load the LLAMA 3.1 base model with the precision reduced from 32 bits to 8 bits.
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

## Restart your session!

In order to load the next model and clear out the cache of the last model, you'll now need to go to Runtime >> Restart session and run the initial cells (imports and HuggingFace login) again.

This is to clean out the GPU.

In [ ]:
# Load the Tokenizer and the Base Model using 4 bit

# The main thing it happens in
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load in th 4 bit quantization.
    bnb_4bit_use_double_quant=True,  # It conducts two passes of the quantizating all of the weights and with that it reduces the memory by 10-20% (Which is good).
    bnb_4bit_compute_dtype=torch.bfloat16, # Using bfloat16 for computation usually increases the speed of the training.
    bnb_4bit_quant_type="nf4") # This helps in interpreting the 4 bit numbers in a better way.

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
base_model

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

In [ ]:
print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
fine_tuned_model

In [ ]:
# Each of the Target Modules has 2 LoRA Adaptor matrices, called lora_A and lora_B
# These are designed so that weights can be adapted by adding alpha * lora_A * lora_B
# Let's count the number of weights using their dimensions:

# See the matrix dimensions above
lora_q_proj = 4096 * 32 + 4096 * 32
lora_k_proj = 4096 * 32 + 1024 * 32
lora_v_proj = 4096 * 32 + 1024 * 32
lora_o_proj = 4096 * 32 + 4096 * 32

# Each layer comes to
lora_layer = lora_q_proj + lora_k_proj + lora_v_proj + lora_o_proj

# There are 32 layers
params = lora_layer * 32

# So the total size in MB is
size = (params * 4) / 1_000_000

# At the end, we are able to use these lower dimensional matrices to have an impact on the bigger architecture. 

# 109MB --> It is the size of the paramters that are fine tuning using the LoRA technique.
print(f"Total number of params: {params:,} and size {size:,.1f}MB")